In [ ]:
# Dependencies and Setup
import pandas as pd
import requests
# import numpy as np
import io
import ipyleaflet
import json
from pathlib import Path
from pprint import pprint
from ipyleaflet import Map, Marker
from geopy.exc import GeocoderTimedOut 
from geopy.geocoders import Nominatim 
from ipywidgets import link, FloatSlider
from branca.colormap import linear
from pandas_geojson import to_geojson

In [ ]:
# Downloading the csv file from your GitHub account
# Make sure the url is the raw version of the file on GitHub
url = "https://raw.githubusercontent.com/brown016340/Project---3/benjamin/data/coords.csv"
download = requests.get(url).content

# Reading the downloaded content and turning it into a pandas dataframe
coords_df = pd.read_csv(io.StringIO(download.decode('utf-8')))

# Printing out the first 5 rows of the dataframe
coords_df.head()

In [ ]:
# Downloading the csv file from your GitHub account
# Make sure the url is the raw version of the file on GitHub
url = "https://raw.githubusercontent.com/brown016340/Project---3/benjamin/data/2015.csv"
download = requests.get(url).content

# Reading the downloaded content and turning it into a pandas dataframe
happy15_df = pd.read_csv(io.StringIO(download.decode('utf-8')))

# Printing out the first 5 rows of the dataframe
happy15_df.head()

In [ ]:
# Join the dataframes
data_df = pd.merge(happy15_df, coords_df, on='Country', how='inner')
data_df.head()

In [ ]:
# Convert to geo json
geo_json = to_geojson(df=data_df, lat='Latitude', lon='Longitude',
                    #   id= 'Country', 
                      properties=['Country','Happiness Rank','Happiness Score'])
pprint(geo_json)

In [ ]:
# Function to add 'id' field to each feature
def add_id_to_features(json_data):
    for feature in json_data['features']:
        # Assuming 'Country' is unique for each feature
        country_name = feature['properties'].get('Country', 'Unknown')
        # Create a simple identifier based on the first three characters of the country name
        feature['id'] = country_name[:2].upper()

# Call the function to add 'id' field
add_id_to_features(geo_json)

# Print the modified JSON
pprint(geo_json)

In [ ]:
happy_score = dict(zip(data_df['Country'].str[:2].str.upper().tolist(), data_df['Happiness Score'].tolist()))

In [ ]:
print(dict(zip(data_df['Country'].str[:2].str.upper().tolist(), data_df['Happiness Score'].tolist())))

In [ ]:
def load_data(url, filename, file_type):
    r = requests.get(url)
    with open(filename, 'w') as f:
        f.write(r.content.decode("utf-8"))
    with open(filename, 'r') as f:
        return file_type(f)

geo_json_data = load_data(
    'https://raw.githubusercontent.com/brown016340/Project---3/benjamin/data/countries.geojson',
    'countries.json',
     json.load)

In [ ]:
pprint(geo_json_data)

In [ ]:
happy_score = dict(zip(data_df['Country'].tolist(), data_df['Happiness Score'].tolist()))

layer = ipyleaflet.Choropleth(
    geo_data= geo_json,
    choro_data=dict(zip(data_df['Country'].str[:2].str.upper().tolist(), data_df['Happiness Score'].tolist())),
    colormap=linear.YlOrRd_04,
    border_color='black',
    style={'fillOpacity': 0.8, 'dashArray': '5, 5'})

m = ipyleaflet.Map(center = (0,0), zoom = 1)
m.add(layer)
m

In [ ]:
admin_country_mapping = {feature['properties']['ADMIN']: feature for feature in geo_json_data['features']}

# Update the geometry in geo_json based on the mapping
for feature in geo_json['features']:
    country_name = feature['properties']['Country']
    if country_name in admin_country_mapping:
        new_geometry = admin_country_mapping[country_name]['geometry']
        feature['geometry'] = new_geometry

In [ ]:
pprint(geo_json)

In [ ]:
layer = ipyleaflet.Choropleth(
    geo_data= geo_json,
    choro_data=dict(zip(data_df['Country'].str[:2].str.upper().tolist(), data_df['Happiness Score'].tolist())),
    colormap=linear.YlOrRd_04,
    border_color='black',
    style={'fillOpacity': 0.8, 'dashArray': '5, 5'})

m = ipyleaflet.Map(center = (0,0), zoom = 1)
m.add(layer)